In [1]:
! pip install statsmodels

You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('pruebillas.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [3]:
import numpy as np
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from backbone.probability_transformer import ProbabilityTransformer 
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from typing import Tuple
import yaml
from sklearn.metrics import classification_report
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.tsa.filters.hp_filter import hpfilter
import os
from sklearn.metrics import accuracy_score
from IPython.display import clear_output

pd.options.display.max_rows = 999

In [4]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

def apply_cusum_filter(raw_price, threshold):
    """
    :param raw_price: (series) of close prices.
    :param threshold: (float) when the abs(change) is larger than the threshold, the
    function captures it as an event.
    :return: (datetime index vector) vector of datetimes when the events occurred. This is used later to sample.
    """
    print('Applying Symmetric CUSUM filter.')

    t_events = []
    s_pos = 0
    s_neg = 0

    # log returns
    diff = np.log(raw_price).diff().dropna()

    # Get event time stamps for the entire series
    for i in tqdm(diff.index[1:]):
        pos = float(s_pos + diff.loc[i])
        neg = float(s_neg + diff.loc[i])
        s_pos = max(0.0, pos)
        s_neg = min(0.0, neg)

        if s_neg < -threshold:
            s_neg = 0
            t_events.append(i)

        elif s_pos > threshold:
            s_pos = 0
            t_events.append(i)

    event_timestamps = pd.DatetimeIndex(t_events)
    return event_timestamps

# Función para calcular la volatilidad diaria
def get_daily_volatility(close_prices, span=100):
    returns = close_prices.pct_change()
    volatility = returns.ewm(span=span).std()
    return volatility

def apply_triple_barrier(
    close_prices, 
    high_prices, 
    low_prices, 
    take_profit_in_pips, 
    stop_loss_in_pips, 
    side,
    max_holding_period=50, 
    pip_size=0.0001
    ):

    barriers = []
    for index in range(len(close_prices)):
        actual_close_price = close_prices[index]
        
        if side[index] == 1:
            # Para una señal de compra
            upper_barrier_level = round(actual_close_price + (take_profit_in_pips * pip_size), 4)
            lower_barrier_level = round(actual_close_price - (stop_loss_in_pips * pip_size), 4)
        elif side[index] == -1:
            # Para una señal de venta
            upper_barrier_level = round(actual_close_price + (stop_loss_in_pips * pip_size), 4)
            lower_barrier_level = round(actual_close_price - (take_profit_in_pips * pip_size), 4)
        else:
            # Si no hay señal, saltar al siguiente índice
            continue
        
        # Evaluar los precios futuros dentro del período máximo de mantenimiento
        for j in range(index + 1, min(index + max_holding_period, len(close_prices))):
            future_close_price = close_prices[j]
            future_high_price = high_prices[j]
            future_low_price = low_prices[j]
            
            if side[index] == 1:

                if future_low_price <= lower_barrier_level:
                    barriers.append((index, 0))  # Etiqueta 0 para stop-loss
                    break

                elif future_high_price >= upper_barrier_level:
                    barriers.append((index, 1))  # Etiqueta 1 para toma de ganancias
                    break

            elif side[index] == -1:
                # Señal de venta: tomar ganancias si se alcanza la barrera inferior

                if future_high_price >= upper_barrier_level:
                    barriers.append((index, 0))  # Etiqueta 0 para stop-loss
                    break

                elif future_low_price <= lower_barrier_level:
                    barriers.append((index, 1))  # Etiqueta 1 para toma de ganancias
                    break

        else:
            barriers.append((index, 2))  # Etiqueta 2 si no se alcanza ninguna barrera
    
    # # Revisar los eventos etiquetados como 2 para determinar si son ganancias o pérdidas
    # for idx, (event_index, label) in enumerate(barriers):
    #     if label == 2:
    #         # Determinar si el precio final fue una ganancia o una pérdida
    #         final_price = close_prices[min(event_index + max_holding_period, len(close_prices) - 1)]
    #         initial_price = close_prices[event_index]
            
    #         if side[event_index] == 1:
    #             # Para una señal de compra
    #             if final_price >= initial_price:
    #                 barriers[idx] = (event_index, 1)  # Etiqueta 1 para toma de ganancias
    #             elif final_price < initial_price:
    #                 barriers[idx] = (event_index, 0)  # Etiqueta 0 para stop-loss
    #         elif side[event_index] == -1:
    #             # Para una señal de venta
    #             if final_price <= initial_price:
    #                 barriers[idx] = (event_index, 1)  # Etiqueta 1 para toma de ganancias
    #             elif final_price > initial_price:
    #                 barriers[idx] = (event_index, 0)  # Etiqueta 0 para stop-loss

    return barriers



def triple_barrier_labeling(
        close_prices, 
        high_prices, 
        low_prices, 
        take_profit_in_pips, 
        stop_loss_in_pips, 
        side,
        max_holding_period=50, 
        pip_size=0.0001,
    ):

    labels = apply_triple_barrier(
        close_prices,
        high_prices, 
        low_prices, 
        take_profit_in_pips, 
        stop_loss_in_pips, 
        side,
        max_holding_period, 
        pip_size
    )
    
    target = [label for _, label in labels]
    return target


In [5]:
def bband_stgy(prices_with_indicators, window):
    df = prices_with_indicators.copy()
    
    df['bband_flag'] = 0
    df['bband_flag'] = np.where((df['Close'] > df['upper_bband']), -1, df['bband_flag']) 
    df['bband_flag'] = np.where((df['Close'] < df['lower_bband']), 1, df['bband_flag']) 

    df['bband_distance_flag'] = 0
    df['bband_distance_flag'] = np.where((df['distance_between_bbands'] > 90), 1, df['bband_distance_flag']) 

    # Bband Flag
    df['bband_flag_negative_window'] = df['bband_flag'].rolling(window=window).apply(lambda x: 1 if (x == -1).all() else 0, raw=True)
    df['bband_distance_flag_window'] = df['bband_distance_flag'].rolling(window=window).apply(lambda x: 1 if (x == 1).all() else 0, raw=True)

    amount_conditions = 3

    short_signals = (
        (df['bband_flag_negative_window'] == 1).astype(int) +
        (df['Close'].shift(-1) < df['upper_bband'].shift(-1)).astype(int) +
        # (df['Close'] < df['upper_bband']).astype(int) +
        (df['bband_distance_flag_window'].shift(-1) == 1).astype(int)
    ) >= amount_conditions

    df.loc[short_signals, 'side'] = -1

    df.dropna(inplace=True)
    return df

In [6]:
symbols_path = './backbone/data/backtest/symbols/USDCHF.csv'
df = pd.read_csv(symbols_path)
df = df[['Date','Close','High','Low','Open', 'upper_bband','lower_bband','macdhist','rsi', 'adx','mfi', 'sma_12', 'sma_26', 'distance_between_bbands']]
print('Creando target')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.set_index('Date')

instrument = df.copy()
# compute bband sides

instrument['side'] = np.nan

window = 3
instrument = bband_stgy(instrument, window)

instrument.dropna(inplace=True)

Creando target


In [7]:
sl = 15
rr = 2
holding_period = 12

instrument['target'] = triple_barrier_labeling(
    close_prices=instrument['Close'], 
    high_prices=instrument['High'], 
    low_prices=instrument['Low'], 
    stop_loss_in_pips=sl, 
    take_profit_in_pips=sl * rr, 
    max_holding_period=holding_period, 
    pip_size=0.001,
    side=instrument['side']
)

df.loc[instrument.index, 'side'] = instrument.side
df.loc[instrument.index, 'target'] = instrument.target
df.fillna(0, inplace=True)
# df.reset_index(inplace=True)


# Aplicar la lógica para mantener solo un 1 o -1 por cada secuencia de 1s o -1s consecutivos
df['side'] = df['side'].where(
    (df['side'] == 0) | (df['side'] != df['side'].shift(1)),
    0
)


df['target'] = np.where(df['side'] != 0, df['target'], 0)

display(pd.DataFrame(df[df.side!=0].groupby(by=['side','target'])['target'].count()))



df['dummy_model_pred'] = 1
print(classification_report(df[df.side != 0].target, df[df.side != 0].dummy_model_pred))

target
side target        
-1.0 0.0         40
     1.0         22
     2.0          6

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        40
         1.0       0.32      1.00      0.49        22
         2.0       0.00      0.00      0.00         6

    accuracy                           0.32        68
   macro avg       0.11      0.33      0.16        68
weighted avg       0.10      0.32      0.16        68



c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [8]:
df = df.reset_index()

In [9]:
df_bt = df[df.Date > '2023-01-01']

In [13]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib


class Bband(Strategy):
    max_hold_period = 12
    hold_period = 0
    sl = 15
    rr = 3
    pips = 0.0001
    
    def init(self):
        # self.side = self.I(SIDE)
        self.upper_bband, self.middle_band, self.lower_bband = self.I(talib.BBANDS, self.data.Close, timeperiod=50)
        close = self.data.Close
        open_ = self.data.Open

    
    def next(self):

        if self.trades:
            self.hold_period += 1

            if self.hold_period > self.max_hold_period:
                self.position.close()
                self.hold_period = 0
        
        lower_bband = self.lower_bband[-1]
        lower_bband_1 = self.lower_bband[-2]
        lower_bband_2 = self.lower_bband[-3]
        lower_bband_3 = self.lower_bband[-4]

        upper_bband = self.upper_bband[-1]
        upper_bband_1 = self.upper_bband[-2]
        upper_bband_2 = self.upper_bband[-3]
        upper_bband_3 = self.upper_bband[-4]

        close = self.data.Close[-1]     
        close_1 = self.data.Close[-2]     
        close_2 = self.data.Close[-3]     
        close_3 = self.data.Close[-4]     

        open_ = self.data.Open[-1]
        open_1 = self.data.Open[-2]
        
        distance_bbands = (self.upper_bband - self.lower_bband) / self.pips
        longer_distance = distance_bbands > 90

        exist_bull_volatility = close_1 > upper_bband_1 and close_2 > upper_bband_2 #and close_3 > upper_bband_3
        volatility_bull_ends = close < open_ and close_2 < open_1

        exist_bear_volatility = close_1 < lower_bband_1 and close_2 < lower_bband_2 #and close_3 > upper_bband_3
        volatility_bear_ends = close > open_ and close_2 > open_1

        # if exist_bear_volatility and longer_distance and volatility_bear_ends and not self.trades:
        #     sl = round(close - (self.sl * self.pips), 4)
        #     tp = round(close + (self.rr * self.sl * self.pips), 4)
        #     self.buy(sl=sl, tp=tp, size=0.1)

        if exist_bull_volatility and longer_distance and volatility_bull_ends and not self.trades:
            sl = round(close + (self.sl * self.pips), 4)
            tp = round(close - (self.rr * self.sl * self.pips), 4)
            self.sell(sl=sl, tp=tp, size=0.1)


In [14]:
from backtesting import Backtest

bt = Backtest(df_bt, Bband, cash=1000, commission=0.0002)
stats = bt.run()
stats

c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
C:\Users\SaidJ\AppData\Local\Temp\ipykernel_20916\2785788255.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_bt, Bband, cash=1000, commission=0.0002)


Start                                  8017.0
End                                   14224.0
Duration                               6207.0
Exposure Time [%]                    1.481959
Equity Final [$]                  1000.065837
Equity Peak [$]                   1000.838444
Return [%]                           0.006584
Buy & Hold Return [%]               -8.980462
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.082232
Avg. Drawdown [%]                   -0.030439
Max. Drawdown Duration                 3634.0
Avg. Drawdown Duration                  864.2
# Trades                                 14.0
Win Rate [%]                        35.714286
Best Trade [%]                        0.46639
Worst Trade [%]                     -0.227299
Avg. Trade [%]                    

In [15]:
bt.plot()


GridPlot(id='p2353', ...)